In [ ]:
# train_dir = 'drive/My Drive/DPPM/train'
# test_dir = 'drive/My Drive/DPPM/test'

In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# def image_gen_w_aug(train_parent_directory, test_parent_directory):
#   train_datagen = ImageDataGenerator(rescale=1/255,
#                                       rotation_range = 30,  
#                                       zoom_range = 0.2, 
#                                       width_shift_range=0.1,  
#                                       height_shift_range=0.1,
#                                       validation_split = 0.15)
  
#   test_datagen = ImageDataGenerator(rescale=1/255)
  
#   train_generator = train_datagen.flow_from_directory(train_parent_directory,
#                                   target_size = (75,75),
#                                   batch_size = 214,
#                                   class_mode = 'categorical',
#                                   subset='training')
  
#   val_generator = train_datagen.flow_from_directory(train_parent_directory,
#                                   target_size = (75,75),
#                                   batch_size = 37,
#                                   class_mode = 'categorical',
#                                   subset = 'validation')
  
#   test_generator = test_datagen.flow_from_directory(test_parent_directory,
#                                  target_size=(75,75),
#                                  batch_size = 37,
#                                  class_mode = 'categorical')
  
#   return train_generator, val_generator, test_generator

In [ ]:
# train_generator, validation_generator, test_generator = image_gen_w_aug(train_dir, test_dir)

In [ ]:
# from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.layers import Flatten, Dense, Dropout

In [ ]:
# def model_output_for_TL (pre_trained_model, last_output):    
#   x = Flatten()(last_output)
  
#   # Dense hidden layer
#   x = Dense(512, activation='relu')(x)
#   x = Dropout(0.2)(x)
    
#   # Output neuron. 
#   x = Dense(3, activation='softmax')(x)
    
#   model = Model(pre_trained_model.input, x)
    
#   return model

# pre_trained_model = InceptionV3(input_shape = (75, 75, 3), 
#                                 include_top = False, 
#                                 weights = 'imagenet')

# for layer in pre_trained_model.layers:
#   layer.trainable = False

# last_layer = pre_trained_model.get_layer('mixed5')
# last_output = last_layer.output
# model_TL = model_output_for_TL(pre_trained_model, last_output)

In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
# import scipy as scipy
# from skimage import data, io, filters

def image_gen_w_aug(train_parent_directory, test_parent_directory):
    
    train_datagen = ImageDataGenerator(rescale=1/255,
                                      rotation_range = 30,  
                                      zoom_range = 0.2, 
                                      width_shift_range=0.1,  
                                      height_shift_range=0.1,
                                      validation_split = 0.15)
    
  
    
    test_datagen = ImageDataGenerator(rescale=1/255)
    
    train_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                       target_size = (75,75),
                                                       batch_size = 214,
                                                       class_mode = 'categorical',
                                                       subset='training')
    
    val_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                          target_size = (75,75),
                                                          batch_size = 37,
                                                          class_mode = 'categorical',
                                                          subset = 'validation')
    
    test_generator = test_datagen.flow_from_directory(test_parent_directory,
                                                     target_size=(75,75),
                                                     batch_size = 37,
                                                     class_mode = 'categorical')
    
    return train_generator, val_generator, test_generator


def model_output_for_TL (pre_trained_model, last_output):

    x = Flatten()(last_output)
    
    # Dense hidden layer
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    # Output neuron. 
    x = Dense(4, activation='softmax')(x)
    
    model = Model(pre_trained_model.input, x)
    
    return model


train_dir = 'malaria/train'
test_dir = 'malaria/test'

train_generator, validation_generator, test_generator = image_gen_w_aug(train_dir, test_dir)

pre_trained_model = InceptionV3(input_shape = (75, 75, 3), 
                                include_top = False, 
                                weights = 'imagenet')

for layer in pre_trained_model.layers:
  layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed3')
last_output = last_layer.output

model_TL = model_output_for_TL(pre_trained_model, last_output)
model_TL.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_TL = model_TL.fit_generator(
      train_generator,
      steps_per_epoch=10,  
      epochs=100,
      verbose=1,
      validation_data = validation_generator)

tf.keras.models.save_model(model_TL,'malaria_model1.hdf5')

Found 134 images belonging to 4 classes.
Found 22 images belonging to 4 classes.
Found 20 images belonging to 4 classes.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Epoch 1/100
10/10 [==============================] - 578s 58s/step - loss: 7.7095 - acc: 0.3567 - val_loss: 3.5129 - val_acc: 0.1818
Epoch 2/100
10/10 [==============================] - 391s 39s/step - loss: 1.0637 - acc: 0.6090 - val_loss: 1.8185 - val_acc: 0.4091
Epoch 3/100
10/10 [==============================] - 252s 25s/step - loss: 0.4628 - acc: 0.8201 - val_loss: 3.3935 - val_acc: 0.2273
Epoch 4/100
10/10 [==============================] - 260s 26s/step - loss: 0.2554 - acc: 0.9157 - val_loss: 3.1258 - val_acc: 0.2727
Epoch 5/100
10/10 [==============================] - 254s 25s/step - loss: 0.1611 - acc: 0.9560 - val_loss: 2.9897 - val_acc: 0.3182
Epoch 6/100
10/10 [==============================] - 253s 25s/step - loss: 0.1081 - acc: 0.9784 - val_loss: 3.6814 - val_acc: 0.2273


10/10 [==============================] - 232s 23s/step - loss: 0.0064 - acc: 0.9993 - val_loss: 6.4916 - val_acc: 0.2273
Epoch 36/100
10/10 [==============================] - 236s 24s/step - loss: 0.0052 - acc: 1.0000 - val_loss: 6.3197 - val_acc: 0.2273
Epoch 37/100
10/10 [==============================] - 238s 24s/step - loss: 0.0069 - acc: 0.9993 - val_loss: 4.4103 - val_acc: 0.2273
Epoch 38/100
10/10 [==============================] - 237s 24s/step - loss: 0.0062 - acc: 1.0000 - val_loss: 5.3881 - val_acc: 0.2273
Epoch 39/100
10/10 [==============================] - 233s 23s/step - loss: 0.0072 - acc: 1.0000 - val_loss: 6.0266 - val_acc: 0.2273
Epoch 40/100
10/10 [==============================] - 229s 23s/step - loss: 0.0058 - acc: 1.0000 - val_loss: 5.9908 - val_acc: 0.2273
Epoch 41/100
10/10 [==============================] - 233s 23s/step - loss: 0.0049 - acc: 1.0000 - val_loss: 5.9000 - val_acc: 0.2273
Epoch 42/100
10/10 [==============================] - 229s 23s/step - loss:

10/10 [==============================] - 230s 23s/step - loss: 0.0020 - acc: 1.0000 - val_loss: 7.7262 - val_acc: 0.2273
Epoch 72/100
10/10 [==============================] - 230s 23s/step - loss: 0.0033 - acc: 1.0000 - val_loss: 7.6198 - val_acc: 0.2273
Epoch 73/100
10/10 [==============================] - 230s 23s/step - loss: 0.0023 - acc: 1.0000 - val_loss: 6.4465 - val_acc: 0.2273
Epoch 74/100
10/10 [==============================] - 230s 23s/step - loss: 0.0024 - acc: 1.0000 - val_loss: 7.1432 - val_acc: 0.2273
Epoch 75/100
10/10 [==============================] - 237s 24s/step - loss: 0.0018 - acc: 1.0000 - val_loss: 8.0646 - val_acc: 0.2273
Epoch 76/100
10/10 [==============================] - 236s 24s/step - loss: 0.0026 - acc: 1.0000 - val_loss: 7.7703 - val_acc: 0.2273
Epoch 77/100
10/10 [==============================] - 232s 23s/step - loss: 0.0020 - acc: 1.0000 - val_loss: 7.0818 - val_acc: 0.2273
Epoch 78/100
10/10 [==============================] - 236s 24s/step - loss:

In [2]:
%%writefile malaria_deploy2.py

import numpy as np
import streamlit as st
import tensorflow as tf
from PIL import Image, ImageOps

def import_and_predict(image_data, model):
    
        size = (75,75)    
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = image.convert('RGB')
        image = np.asarray(image)
        image = (image.astype(np.float32) / 255.0)

        img_reshape = image[np.newaxis,...]

        prediction = model.predict(img_reshape)
        
        return prediction

model = tf.keras.models.load_model('malaria_model1.hdf5')

st.write("""
         # Malaria Virus Platform Prediction
         """
         )

st.write("This is a simple image classification web app to predict Malaria Virus")

file = st.file_uploader("Please upload an image file", type=["jpg", "png"])
#
if file is None:
    st.text("You haven't uploaded an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    prediction = import_and_predict(image, model)
    
    if np.argmax(prediction) == 0:
        st.write("It is a Falciparum")
    elif np.argmax(prediction) == 1:
        st.write("It is a Malariae")
    elif np.argmax(prediction) == 2:
        st.write("it ia a Ovale")
    else:
        st.write("It is a Vivax")
    
    st.text("Probability (0: Falciparum, 1: Malariae, 2: Ovale, 3: Vivax)")
    st.write(prediction)

Writing malaria_deploy2.py
